# Import Libraies

In [2]:
# %pip install selenium
# %pip install pymongo

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# import some basic python libaries (operating system, regular expressions, time)
import os
import re
import time
from datetime import datetime
import random
# import some selenium components to rememeneber by name
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException

# Import Load login credentials from .env file, make sure to pip install python-dotenv
#import mongoDB, we may add additional databases in the future for better 
from selenium_stealth import stealth

from pymongo import MongoClient

from bs4 import BeautifulSoup



### Load Enviroment variables, configure web driver, and create database connection

In [4]:
#load user login credentials from .env file, not needed for indeed
# load_dotenv()
# username = os.getenv("USERNAME")
# password = os.getenv("PASSWORD")

# Possibly test regex if we want to make  site for cs only
regexPattern=r'\b(?:devOPS|auto|Automation|java|web development|python|javascript)\b' #

#
job_count = 0
# Create a MongoClient to the running mongod instance
client = MongoClient("mongodb://localhost:27017")
db = client["my_database"]
#create collection to stoer data
collection = db['opportunities']

#current tie for log name
timestamp = datetime.now().strftime('%Y/%m/%d_%H-%M-%S')


In [5]:
#DB testing
data = {"description": "This is a test description", "date": datetime.now()}
result= collection.insert_one(data)
db.list_collection_names()
print(collection.find_one())

{'_id': ObjectId('66595231d8b54b50016ecb0e'), 'description': 'This is a test description', 'date': datetime.datetime(2024, 5, 31, 0, 29, 37, 224000)}


# Functions

In [6]:

def write_to_log(text):
    """
    Writes the given text to a log file specified by filepath.
    Parameters:
    filepath (str): The path to the log file.
    text (str): The text to write to the log file.
    """

     # Ensure the 'logs' directory exists

    # Get the current working directory
    current_dir = os.getcwd()
    
    # Ensure the 'logs' directory exists in the current working directory
    log_dir = os.path.join(current_dir, 'logs')
    os.makedirs(log_dir, exist_ok=True)
    
    # Generate a timestamped filename
    log_filename = f"log_{timestamp}.log"
    log_filepath = os.path.join(log_dir, log_filename)

    
    try:
        with open(log_filepath, 'a') as log_file:
            log_file.write(text + '\n')
    except Exception as e:
        print(f"An error occurred while writing to the log file: {e}")


def close_modal_popup(driver, timeout=5):
    """
    Close the modal popup if it appears.

    Parameters:
    - driver: The WebDriver instance.
    - timeout: Maximum time to wait for the modal popup to appear (default is 10 seconds).
    """
    try:
        # Wait for the modal to be visible and the close button to be clickable
        WebDriverWait(driver, timeout).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'button[aria-label="close"]'))
        )
        close_button = WebDriverWait(driver, timeout).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="close"]'))
        )
        
        # Click the close button
        close_button.click()
        print("Modal popup closed.")
    except Exception as e:
        print(f"Modal popup did not appear or could not be closed: {e}")
        

def try_and_fix_cloud_flare(driver, timeout=10):
    """
    Try to fix Cloudflare issues by refreshing the page and waiting for the page to load.

    Parameters:
    - driver: The WebDriver instance.
    - timeout: Maximum time to wait for the page to load (default is 10 seconds).
    """
    try:
        time.sleep(1)
        close_modal_popup(driver)
        # Refresh the page
        # if driver.forward():
        driver.forward()
        print("Page forward.")
        # else:
        #     driver.back()
        #     print("Page back.")

        close_modal_popup(driver)
        
        # Wait for the page to load
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a[aria-label='Next Page']"))
        )
        print("Page loaded successfully.")
    except Exception as e:
        print(f"Cloudflare issue could not be fixed: {e}")



        #---------------- RANDOM FUCNTIONS
def random_iteration(elements):
            # Shuffle the elements randomly
            random.shuffle(elements)
            
            # Iterate over the shuffled elements
            for element in elements:
                # Process the element
                print(element)
                
                # Check if all elements have been found
                if len(set(elements)) == len(elements):
                    break

#random wait function
def random_wait():
    random_time = random.randint(1, 3)/2 + random.randint(1, 3)/2


## Open the Website and scrape the data

### alt links
 - https://ca.indeed.com/jobs?q=intern&l=Canada&start=30&vjk=a092852315dc980f

In [7]:
#open the website
#open all the job postings and save the data+ links to mondoDB 
# DO ONE, then do this in a loop until we reach the end and thats it
#jcs-JobTitle css-jspxzf eu4oa1w0
try:
    # Open the website
    # setup cloudflare protection
    
    #Configuest Selenium stelath

    # Add user-agent to the webdriver options
    user_agents = [
        # Your list of user agents goes here
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36", 
	"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36", 
    ]
    user_agent = random.choice(user_agents)
    
    options = webdriver.ChromeOptions()
    options.add_argument(f"user-agent={user_agent}")

    # Initialize the WebDriver with options
    driver = webdriver.Chrome(options=options)

    # more attempts
    # Adding argument to disable the AutomationControlled flag 
    options.add_argument("--disable-blink-features=AutomationControlled") 
    
    # Exclude the collection of enable-automation switches 
    options.add_experimental_option("excludeSwitches", ["enable-automation"]) 
    
    # Turn-off userAutomationExtension 
    options.add_experimental_option("useAutomationExtension", False) 

    # Apply stealth settings to the driver
    stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
            )
    #initialize webdriver
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})") 

    
    driver.get(r"https://ca.indeed.com/jobs?l=Canada&sc=0kf%3Ajt%28internship%29%3B&vjk=761a620c59aad22c")
    
    next_page_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[aria-label='Next Page']")))

    # Find the next page button and loop based on its pressance
    while True: #put a better loop condition here

        # Find all the job posting links on this page, randomize their order with shuffle?
        all_position_links = driver.find_elements(By.XPATH, '//td[@class="resultContent css-1qwrrf0 eu4oa1w0"]//a')
        random.shuffle(all_position_links)

        # Iterate through each link
        for index, link in enumerate(all_position_links):
            try:
                job_count= job_count+ 1
                
                # Click the link
                ActionChains(driver).scroll_to_element(link).perform()
                link.click()
                random_wait()
                time.sleep(2)
                #
                
                # big_body = WebDriverWait(driver, 5).until(
                #     EC.presence_of_element_located((By.CSS_SELECTOR, ".jobsearch-JobComponent-description.css-10ybyod.eu4oa1w0"))
                # )
                job_description_section = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, 'jobDescriptionText')))

                #job_description_html = job_description_section.get_attribute('innerHTML')
                # If you need plain text instead of HTML, you can use .text
                job_description_text = job_description_section.text
                # write_to_log(job_description_text)
                data = {"description": job_description_text, "date": datetime.now()}
                collection.insert_one(data)
                print("Job saved successfully!")
                # Navigate back to the main listings page before clicking the next link
                driver.back()

            except TimeoutException as e:
                print(f"Timeout Exception for job {job_count}: Could not find the salary information.")
                driver.back()
            
            except NoSuchElementException as e:
                print(f"NoSuchElementException for job {job_count}: Could not find an element.")
                driver.back()
            
            except WebDriverException as e:
                print(f"WebDriverException for job {job_count}: An issue with the WebDriver.")
                driver.back()
            
            except Exception as e:
                print(f"An unexpected error occurred for job {job_count}.")
                print(e)
                driver.back()

        #Eachloop iteration go tothe next page
        try_and_fix_cloud_flare(driver,5)
        next_page_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[aria-label='Next Page']")))

finally:
    # Close the browser after processing all links
    print("Finnaly reached!")

driver.quit()
print("Success!")



Job saved successfully!
Job saved successfully!
Job saved successfully!
Job saved successfully!
Job saved successfully!
Job saved successfully!
Job saved successfully!
Job saved successfully!
Job saved successfully!
Job saved successfully!
Job saved successfully!
Job saved successfully!
Job saved successfully!
Job saved successfully!
Job saved successfully!
Modal popup did not appear or could not be closed: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF613B33E32+31618]
	(No symbol) [0x00007FF613AAB099]
	(No symbol) [0x00007FF61396888A]
	(No symbol) [0x00007FF6139B8524]
	(No symbol) [0x00007FF6139B862C]
	(No symbol) [0x00007FF6139FF787]
	(No symbol) [0x00007FF6139DD14F]
	(No symbol) [0x00007FF6139FCA80]
	(No symbol) [0x00007FF6139DCEB3]
	(No symbol) [0x00007FF6139AA46B]
	(No symbol) [0x00007FF6139AB001]
	GetHandleVerifier [0x00007FF613E39FFD+3202381]
	GetHandleVerifier [0x00007FF613E86A1D+3516269]
	GetHandleVerifier [0x00007FF613E7C490+3473888]
	GetHandleVerifier [0x00007FF613BE5D3

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.63)
Stacktrace:
	GetHandleVerifier [0x00007FF613B33E32+31618]
	(No symbol) [0x00007FF613AAB099]
	(No symbol) [0x00007FF61396888A]
	(No symbol) [0x00007FF61393DAE5]
	(No symbol) [0x00007FF6139E45A7]
	(No symbol) [0x00007FF6139FC131]
	(No symbol) [0x00007FF6139DCEB3]
	(No symbol) [0x00007FF6139AA46B]
	(No symbol) [0x00007FF6139AB001]
	GetHandleVerifier [0x00007FF613E39FFD+3202381]
	GetHandleVerifier [0x00007FF613E86A1D+3516269]
	GetHandleVerifier [0x00007FF613E7C490+3473888]
	GetHandleVerifier [0x00007FF613BE5D36+760454]
	(No symbol) [0x00007FF613AB6B3F]
	(No symbol) [0x00007FF613AB1CD4]
	(No symbol) [0x00007FF613AB1E62]
	(No symbol) [0x00007FF613AA120F]
	BaseThreadInitThunk [0x00007FFE0F7C257D+29]
	RtlUserThreadStart [0x00007FFE10B8AF28+40]


In [ ]:
#TESTING 
#line under here should be turned into a function that auto closes modal popups 
#or this text "I want to receive the latest job alert in Canada"
#div[role='dialog'], div button[aria-label='close']
print(all_position_links)

[<selenium.webdriver.remote.webelement.WebElement (session="be52ef9136f025cf071e72f85b15f874", element="f.0C826D23F1A5E60A128DFF7EFBE1B6C6.d.E4C447BB797B9FFEC3841C05958F46AA.e.5")>, <selenium.webdriver.remote.webelement.WebElement (session="be52ef9136f025cf071e72f85b15f874", element="f.0C826D23F1A5E60A128DFF7EFBE1B6C6.d.E4C447BB797B9FFEC3841C05958F46AA.e.44")>, <selenium.webdriver.remote.webelement.WebElement (session="be52ef9136f025cf071e72f85b15f874", element="f.0C826D23F1A5E60A128DFF7EFBE1B6C6.d.E4C447BB797B9FFEC3841C05958F46AA.e.45")>, <selenium.webdriver.remote.webelement.WebElement (session="be52ef9136f025cf071e72f85b15f874", element="f.0C826D23F1A5E60A128DFF7EFBE1B6C6.d.E4C447BB797B9FFEC3841C05958F46AA.e.46")>, <selenium.webdriver.remote.webelement.WebElement (session="be52ef9136f025cf071e72f85b15f874", element="f.0C826D23F1A5E60A128DFF7EFBE1B6C6.d.E4C447BB797B9FFEC3841C05958F46AA.e.47")>, <selenium.webdriver.remote.webelement.WebElement (session="be52ef9136f025cf071e72f85b15f87